# Xedocs user guide

This tutorial serves as a basic introduction to xedocs. Xedocs is the replacement of CMT (Correction Management System) and will be used moving forward to store, save and access correction data as well as other metadata needed for the XENONnT experiment moving forward. The main goal with xedocs was to have a system with the following requierments: We want versioned data that is flexible enough to adapt to future changes, to have insertion rules and to follow time dependace requirements. 

In [1]:
import strax
import straxen
import xedocs as xd
import numpy as np
import rframe
import pymongo
import matplotlib.pyplot as plt

Could not import editors, GUI not available.


In [2]:
straxen.print_versions('strax straxen rframe xedocs'.split())

,module,version,path,git
0,python,3.8.13,/opt/conda/envs/strax/bin/python,None
1,strax,1.4.0,/home/luissanchez25/.local/lib/python3.8/site-...,branch:relative_light_yield_schema | 3cd255b
2,straxen,2.0.3,/home/luissanchez25/.local/lib/python3.8/site-...,branch:relative_light_yield_schema | 3cd255b
3,rframe,0.2.1,/home/luissanchez25/.local/lib/python3.8/site-...,branch:relative_light_yield_schema | 3cd255b
4,xedocs,0.2.8,/home/luissanchez25/.local/lib/python3.8/site-...,branch:relative_light_yield_schema | 3cd255b


With xedocs we can use schemas to get different corrections from multiple sources such as bodega (refered to as 'detector numbers'), the xedocs databse, or even your own! Schemas are python classes with certain properties and functions to deal with the backend of storing data as well as rules such as preventing the deletion and overwritting of existing data!
<br>
Lets look at some of the schemas that are available!

In [3]:
xd.list_schemas()

['detector_numbers',
 'context_configs',
 'plugin_lineages',
 'context_lineages',
 'fax_configs',
 'electron_drift_velocities',
 'electron_drift_time_gates',
 'electron_lifetimes',
 'rel_extraction_effs',
 'fdc_maps',
 'hit_thresholds',
 'pmt_gains',
 'posrec_models',
 's1_aft_xyz_maps',
 's1_xyz_maps',
 's2_xy_maps',
 'se_gains',
 'electron_diffusion_cte',
 'baseline_samples_nv',
 'relative_light_yield',
 'avg_se_gains',
 'bayes_models',
 'utube_calibrations',
 'diffused_calibrations',
 'ibelt_calibrations',
 'calibration_sources',
 'hotspot_reports',
 'anode_ramps',
 'anode_washes',
 'abnormal_daq_rates',
 'processing_requests',
 'pmt_gain_calculations',
 'pmt_voltage_changes',
 'pmt_installs',
 'pmt_voltage_settings']

### Accessing Data

The development database is a database in which everyone can enter their own corrections, it will also have all the values for each correction. To avoid confusion make sure to label the version as something that cannot possibly be a real correction, here I will use test* 

In [4]:
xd_db_dev = xd.development_db.utilix

The straxen database on the other had will have the real value of all correction and data can only be insterted into this database after other memebers of the colaboration have agreed that his value should be inserted.

In [5]:
xd_db_stx = xd.straxen_db.utilix

The data can be returned to the user in a varaiety of formats, for example in a pandas dataframe structure:

#### Dataframe format

In [6]:
xd_db_stx.electron_lifetimes.find_df(version='ONLINE')

created_date comments reviews  \
version time                                                           
ONLINE  2017-01-01 00:00:00 2022-12-22 15:14:06.975               []   
        2020-10-14 00:00:00 2022-12-22 15:14:06.975               []   
        2020-10-14 06:00:00 2022-12-22 15:14:06.975               []   
        2020-10-14 12:00:00 2022-12-22 15:14:06.975               []   
        2020-10-14 18:00:00 2022-12-22 15:14:06.975               []   
...                                             ...      ...     ...   
        2022-09-30 03:06:03 2022-12-22 15:14:06.971               []   
        2022-09-30 09:06:03 2022-12-22 15:14:06.971               []   
        2022-09-30 15:06:03 2022-12-22 15:14:06.971               []   
        2022-09-30 21:06:03 2022-12-22 15:14:06.971               []   
        2022-10-01 03:06:03 2022-12-22 15:14:06.971               []   

                                    value  
version time                               
ONLINE  2017-01-01 00:00:00  2.000000e+05  
        2020-10-14 00:00:00  7.125580e+04  
        2020-10-14 06:00:00  6.927400e+04  
        2020-10-14 12:00:00  6.903980e+04  
        2020-10-14 18:00:00  6.819510e+04  
...                                   ...  
        2022-09-30 03:06:03  8.180421e+06  
        2022-09-30 09:06:03  8.180421e+06  
        2022-09-30 15:06:03  8.180421e+06  
        2022-09-30 21:06:03  8.180421e+06  
        2022-10-01 03:06:03  8.180421e+06  

[2660 rows x 4 columns]

In [7]:
elife = xd_db_stx.electron_lifetimes.find_df(version='ONLINE') # if you try to 

In [8]:
elife

created_date comments reviews  \
version time                                                           
ONLINE  2017-01-01 00:00:00 2022-12-22 15:14:06.975               []   
        2020-10-14 00:00:00 2022-12-22 15:14:06.975               []   
        2020-10-14 06:00:00 2022-12-22 15:14:06.975               []   
        2020-10-14 12:00:00 2022-12-22 15:14:06.975               []   
        2020-10-14 18:00:00 2022-12-22 15:14:06.975               []   
...                                             ...      ...     ...   
        2022-09-30 03:06:03 2022-12-22 15:14:06.971               []   
        2022-09-30 09:06:03 2022-12-22 15:14:06.971               []   
        2022-09-30 15:06:03 2022-12-22 15:14:06.971               []   
        2022-09-30 21:06:03 2022-12-22 15:14:06.971               []   
        2022-10-01 03:06:03 2022-12-22 15:14:06.971               []   

                                    value  
version time                               
ONLINE  2017-01-01 00:00:00  2.000000e+05  
        2020-10-14 00:00:00  7.125580e+04  
        2020-10-14 06:00:00  6.927400e+04  
        2020-10-14 12:00:00  6.903980e+04  
        2020-10-14 18:00:00  6.819510e+04  
...                                   ...  
        2022-09-30 03:06:03  8.180421e+06  
        2022-09-30 09:06:03  8.180421e+06  
        2022-09-30 15:06:03  8.180421e+06  
        2022-09-30 21:06:03  8.180421e+06  
        2022-10-01 03:06:03  8.180421e+06  

[2660 rows x 4 columns]

#### list format

In [17]:
elife = xd_db_stx.electron_lifetimes.find(version='ONLINE') # if you try to 

In [18]:
list(elife)[:3]

[{'version': 'ONLINE',
  'created_date': datetime.datetime(2022, 12, 22, 15, 14, 6, 975000),
  'comments': '',
  'reviews': [],
  'time': datetime.datetime(2017, 1, 1, 0, 0),
  'value': 200000.0},
 {'version': 'ONLINE',
  'created_date': datetime.datetime(2022, 12, 22, 15, 14, 6, 975000),
  'comments': '',
  'reviews': [],
  'time': datetime.datetime(2020, 10, 14, 0, 0),
  'value': 71255.79834},
 {'version': 'ONLINE',
  'created_date': datetime.datetime(2022, 12, 22, 15, 14, 6, 975000),
  'comments': '',
  'reviews': [],
  'time': datetime.datetime(2020, 10, 14, 6, 0),
  'value': 69274.00208}]

#### Dict format

In [20]:
elife = xd_db_stx.electron_lifetimes.find_dicts(version='ONLINE') # if you try to 

In [22]:
elife[:3]

[{'version': 'ONLINE',
  'created_date': datetime.datetime(2022, 12, 22, 15, 14, 6, 975000),
  'comments': '',
  'reviews': [],
  'time': datetime.datetime(2017, 1, 1, 0, 0),
  'value': 200000.0},
 {'version': 'ONLINE',
  'created_date': datetime.datetime(2022, 12, 22, 15, 14, 6, 975000),
  'comments': '',
  'reviews': [],
  'time': datetime.datetime(2020, 10, 14, 0, 0),
  'value': 71255.79834},
 {'version': 'ONLINE',
  'created_date': datetime.datetime(2022, 12, 22, 15, 14, 6, 975000),
  'comments': '',
  'reviews': [],
  'time': datetime.datetime(2020, 10, 14, 6, 0),
  'value': 69274.00208}]

#### "Docs" format

In [24]:
elife = xd_db_stx.electron_lifetimes.find_docs(version='ONLINE') # if you try to 

In [26]:
elife[:3]

[
         Xenon ElectronLifetime Document
         -------------------------------
  
         Category:      corrections
         Alias:         electron_lifetimes
         Index:         version=ONLINE, time=2017-01-01 00:00:00
         Values:        created_date=2022-12-22 15:14:06.975000, comments=, reviews=[], value=200000.0
         ,
 
         Xenon ElectronLifetime Document
         -------------------------------
  
         Category:      corrections
         Alias:         electron_lifetimes
         Index:         version=ONLINE, time=2020-10-14 00:00:00
         Values:        created_date=2022-12-22 15:14:06.975000, comments=, reviews=[], value=71255.79834
         ,
 
         Xenon ElectronLifetime Document
         -------------------------------
  
         Category:      corrections
         Alias:         electron_lifetimes
         Index:         version=ONLINE, time=2020-10-14 06:00:00
         Values:        created_date=2022-12-22 15:14:06.975000, comments=, 

#### You can just get one data point if that is all you want!

In [28]:
elife = xd_db_stx.electron_lifetimes.find_one(version='ONLINE') # if you try to 

In [29]:
elife


        Xenon ElectronLifetime Document
        -------------------------------
 
        Category:      corrections
        Alias:         electron_lifetimes
        Index:         version=ONLINE, time=2017-01-01 00:00:00
        Values:        created_date=2022-12-22 15:14:06.975000, comments=, reviews=[], value=200000.0
        

##### You can pass many different arguments into the search parameters to make your scope as limited or as wide as you might want it, you can also use _sort options if you want the data to be returned in a particualr order.

In [31]:
rel_ext_eff = xd_db_stx.rel_extraction_effs.find_docs(version='v3')

In [39]:
rel_ext_eff[:3]

[
         Xenon RelExtractionEff Document
         -------------------------------
  
         Category:      corrections
         Alias:         rel_extraction_effs
         Index:         version=v3, time=2017-01-01 00:00:00, partition=ab
         Values:        created_date=2022-08-05 15:07:51.600000, comments=, reviews=[], value=1.0
         ,
 
         Xenon RelExtractionEff Document
         -------------------------------
  
         Category:      corrections
         Alias:         rel_extraction_effs
         Index:         version=v3, time=2021-05-03 11:13:17.786000, partition=ab
         Values:        created_date=2022-08-05 15:07:51.600000, comments=, reviews=[], value=1.0
         ,
 
         Xenon RelExtractionEff Document
         -------------------------------
  
         Category:      corrections
         Alias:         rel_extraction_effs
         Index:         version=v3, time=2021-05-03 11:13:17.786000, partition=ab
         Values:        created_date=2022-

In [45]:
# lets sort by time and only get the ab partition
rel_ext_eff = xd_db_stx.rel_extraction_effs.find_docs(version='v3', partition='ab', __sort='time')

In [46]:
rel_ext_eff[-3:]

[
         Xenon RelExtractionEff Document
         -------------------------------
  
         Category:      corrections
         Alias:         rel_extraction_effs
         Index:         version=v3, time=2021-12-30 18:03:53.961000, partition=ab
         Values:        created_date=2022-08-05 15:07:51.352000, comments=, reviews=[], value=0.9999798683656651
         ,
 
         Xenon RelExtractionEff Document
         -------------------------------
  
         Category:      corrections
         Alias:         rel_extraction_effs
         Index:         version=v3, time=2021-12-30 18:18:55.465000, partition=ab
         Values:        created_date=2022-08-05 15:07:51.352000, comments=, reviews=[], value=0.9999798683656651
         ,
 
         Xenon RelExtractionEff Document
         -------------------------------
  
         Category:      corrections
         Alias:         rel_extraction_effs
         Index:         version=v3, time=2022-05-06 21:45:38.508000, partition=ab
     

In [49]:
rel_ext_eff = xd_db_stx.rel_extraction_effs.find_docs(version='v3', 
                                                  run_id = '027434', 
                                                  partition = 'ab')

In [50]:
rel_ext_eff

[
         Xenon RelExtractionEff Document
         -------------------------------
  
         Category:      corrections
         Alias:         rel_extraction_effs
         Index:         version=v3, time=2021-08-30 13:31:05.901000, partition=ab
         Values:        created_date=2022-08-05 15:07:51.377000, comments=, reviews=[], value=1.0
         ]

##### Data stored in xedocs have multiple indecies, and there is a requierment that all indecies are unique, therefore you can only save new data when it has indecies where the combination of all indecies is unique

We can also access the Bodega data through schemas using 'DetectorNumber'

In [52]:
xd_db_stx.detector_numbers.find_one(field='g1')


        Xenon DetectorNumber Document
        -----------------------------
 
        Category:      general
        Alias:         detector_numbers
        Index:         version=v0, field=g1, partition=all_tpc
        Values:        value=0.17, uncertainty=0.0062, definition=, reference=https://github.com/XENONnT/bbf/blob/master/bbf/config/template_config/pars/ERTemplateParameters_XENONnT.ini, date=2021-05-05 00:00:00, comments=Taken from bbf config
        

In [55]:
se_gain = xd_db_stx.detector_numbers.find(field = 'se_gain')

In [56]:
list(se_gain)

[{'version': 'v0',
  'field': 'se_gain',
  'partition': 'all_tpc',
  'value': 32.3604,
  'uncertainty': 0.077,
  'definition': '',
  'reference': 'https://xe1t-wiki.lngs.infn.it/doku.php?id=jlong:sr0_ramp_up_kr_se_study',
  'date': datetime.datetime(2021, 11, 3, 0, 0),
  'comments': 'Fitted from SE population'},
 {'version': 'v1',
  'field': 'se_gain',
  'partition': 'ab',
  'value': 30.090992211040312,
  'uncertainty': 0.08738706743911372,
  'definition': '',
  'reference': 'https://xe1t-wiki.lngs.infn.it/doku.php?id=xenon:xenonnt:analysis:sr0_se_conclusive_page',
  'date': datetime.datetime(2022, 5, 12, 0, 0),
  'comments': 'Poisson&Gaussian fitted from SE population for background_linked runs under sr0_preliminary condition between 2021-07-05 and 2021-08-08'},
 {'version': 'v1',
  'field': 'se_gain',
  'partition': 'all_tpc',
  'value': 31.151821493235058,
  'uncertainty': 0.9715038932217172,
  'definition': '',
  'reference': 'https://xe1t-wiki.lngs.infn.it/doku.php?id=xenon:xenonn

### Saving data using xedocs

To save new data using xedocs we need to know what indecies that particular schema has. By defualt all schemas will require a version and a run_id or time for the indecies and a value to actually upload. However other shcemas can have additional indecies you need to spesify such as algorithm for machine learning related currections such as 'mlp', 'cnn' and 'gcn', format of the data, among others. As such we need to first know what are the indecies required and then we can upload the data!

For this set of correction we will save the data to the development db.

In [61]:
run_id = '047493'

In [63]:
# Get the ONLINE of elife for out run_id
elife_online = xd_db_stx.electron_lifetimes.find_docs(version = 'ONLINE', run_id = run_id)

In [66]:
list(xd_db_dev.electron_lifetimes.find(run_id = run_id))

[]

In [78]:
elife_new = xd.schemas.ElectronLifetime(value=elife_online[0].value*0.90, version='test*', 
                                        run_id=run_id, datasource = 'development_db',
                                        comments='giving a 10% decrease to Electron lifetime')

In [79]:
xd_db_dev.electron_lifetimes.insert(elife_new)

[
         Xenon ElectronLifetime Document
         -------------------------------
  
         Category:      corrections
         Alias:         electron_lifetimes
         Index:         version=test*, time=2022-09-13 11:14:46.741000
         Values:        created_date=2023-01-24 17:20:42.436000, comments=giving a 10% decrease to Electron lifetime, reviews=[], value=7362379.160154001
         ]

In [80]:
list(xd_db_dev.electron_lifetimes.find(run_id = run_id))

[{'version': 'test*',
  'created_date': datetime.datetime(2023, 1, 24, 17, 20, 42, 436000),
  'comments': 'giving a 10% decrease to Electron lifetime',
  'reviews': [],
  'time': datetime.datetime(2022, 9, 13, 11, 14, 46, 741000),
  'value': 7362379.160154001}]

In [81]:
elife_new = xd.schemas.ElectronLifetime(value=elife_online[0].value*0.80, version='test*2', 
                                        run_id=run_id, datasource = 'development_db',
                                        comments='giving a 10% decrease to Electron lifetime')

In [82]:
# there is another way to save this data
xd.insert_docs('electron_lifetimes', elife_new, 'development_db')

[
         Xenon ElectronLifetime Document
         -------------------------------
  
         Category:      corrections
         Alias:         electron_lifetimes
         Index:         version=test*2, time=2022-09-13 11:14:46.741000
         Values:        created_date=2023-01-24 17:20:42.436000, comments=giving a 10% decrease to Electron lifetime, reviews=[], value=6544337.031248
         ]

In [83]:
list(xd_db_dev.electron_lifetimes.find(run_id = run_id))

[{'version': 'test*',
  'created_date': datetime.datetime(2023, 1, 24, 17, 20, 42, 436000),
  'comments': 'giving a 10% decrease to Electron lifetime',
  'reviews': [],
  'time': datetime.datetime(2022, 9, 13, 11, 14, 46, 741000),
  'value': 7362379.160154001},
 {'version': 'test*2',
  'created_date': datetime.datetime(2023, 1, 24, 17, 20, 42, 436000),
  'comments': 'giving a 10% decrease to Electron lifetime',
  'reviews': [],
  'time': datetime.datetime(2022, 9, 13, 11, 14, 46, 741000),
  'value': 6544337.031248}]

#### Now to save data in our own database!

You will not have access to the config file bellow, however if you have mongoDB installed locally you can just ommit the the host, username and password information and just connect to your local mongo_db. You can also call the database whatever you want.

In [94]:
import config

host = config.mongo_rdb_url
username = config.mongo_rdb_username
password = config.mongo_rdb_password

In [110]:
db = pymongo.MongoClient(host = host,
                        username = username,
                        password = password)['correctionsSandbox']

In [112]:
xd.schemas.ElectronLifetime.find(version = 'test*', datasource = db['electron_lifetimes'])

[]

In [113]:
new_elife = xd.schemas.ElectronLifetime(value = 123456, version = 'test*', run_id = run_id)

In [114]:
new_elife.save(db['electron_lifetimes'])

In [115]:
xd.schemas.ElectronLifetime.find(version = 'test*', datasource = db['electron_lifetimes'])

[
         Xenon ElectronLifetime Document
         -------------------------------
  
         Category:      corrections
         Alias:         electron_lifetimes
         Index:         version=test*, time=2022-09-13 11:14:46.741000
         Values:        created_date=2023-01-24 17:20:42.436000, comments=, reviews=[], value=123456.0
         ]

Success! we have inserted data into our own mongo database!